<a href="https://colab.research.google.com/github/jamg-upv/LLMforSLRscreening/blob/main/py/Reto21d_ago24_LLMhiwp_embeddingsv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En principio este codigo funciona con un entorno de de computacion de 12GB RAM sin GPU.
si se atascara por RAM puedo probarlo a partirlo en bloques y que vaya actualizando un fichero de resultados
En ultima instancia me planteo compara bloques de computación para poder usar un GPU o entornos más intensivos de RAM

#Preparacion previa (ejecutar solo una vez)


Conexion a espacios

In [2]:
#conectar con repositorio github publico para acceso a los datasets
!git clone https://github.com/jamg-upv/LLMforSLRscreening.git

#conecar a google drive para guardar alli los outputs que quiera mantener (y que no se me olvide descargarlos) luego los subiré a Git para su uso posterior
from google.colab import drive
import os

# Montar Google Drive si aún no está montado
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

Cloning into 'LLMforSLRscreening'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 114 (delta 50), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (114/114), 1.96 MiB | 5.86 MiB/s, done.
Resolving deltas: 100% (50/50), done.


Parametrización de variables

In [4]:
# Define el nombre del archivo y el path de salida
input_file_name = 'hiwp_componentsv3.csv'
# estructura del archivo de datos:
# Component	Description	Type	Class

input_path = '/content/LLMforSLRscreening/datasets/'

output_path = '/content/drive/MyDrive/Reto21dias_24/'

# Construye el path completo
full_in_path = os.path.join(input_path, input_file_name)
full_in_path
# full_out_path = os.path.join(output_path, file_name)

# Define la variable con el nombre de la columna
target_column = 'Description'

# Objetos a clasificar
selec_objects = ['art']
# selec_objects = ['cat1']

#categorias en la que quiero clasificar los objetos
selec_categories = ['cat1','cat2', 'cat3','cat4','cat5']
# selec_categories = ['cat2', 'cat3','cat4']

# el identificador [component] de la categoria que quiero mostrar en el resumen de clasificación global
target_category ='HIWPshortDescrip'
# target_category ='HIWPLongDescrip'

# etiquetar los archivos con alguna información adcional
# tag_file = ''
tag_file = 'all models-articles'
# tag_file = 'best7 models'

Cargar el archivo de datos y filtrar los objetos a clasificar y las categorias a utilizar para la clasificación

Crea el DataFrame objects filtrando hiwp_data para incluir solo las filas donde el valor en la columna 'Type' está en la lista selec_objects.
Crea el DataFrame categories filtrando hiwp_data para incluir solo las filas donde el valor en la columna 'Type' está en la lista selec_categories.

Si necesito que la comparación no sea sensible a mayúsculas/minúsculas, puedes modificar el código así:

```
objects = hiwp_data[hiwp_data['Type'].str.lower().isin([x.lower() for x in selec_objects])]
categories = hiwp_data[hiwp_data['Type'].str.lower().isin([x.lower() for x in selec_categories])]
```



In [5]:
import pandas as pd
# Leer el archivo CSV (es un CSV de Open Office si lo genero con MSexcel no se mapea bien sin dar atributos adicionales)
hiwp_data = pd.read_csv(full_in_path)

# Mostrar el DataFrame resultante
# print(hiwp_data)

# Crear el DataFrame 'objects'
objects = hiwp_data[hiwp_data['Type'].isin(selec_objects)]

# Crear el DataFrame 'categories'
categories = hiwp_data[hiwp_data['Type'].isin(selec_categories)]

# Opción 1: Reconstruir el índice descartando el índice original
# objects= objects.reset_index(drop=True)
# categories= categories.reset_index(drop=True)

# Opción 2: Reconstruir el índice manteniendo el índice original como una nueva columna
objects= objects.reset_index()
categories= categories.reset_index()


# Verificar los resultados
print("DataFrame 'objects':")
print(objects.shape)
print(objects['Type'].value_counts())
print("\nPrimeras filas de 'objects':")
print(objects.head())

print("\n" + "="*50 + "\n")

print("DataFrame 'categories':")
print(categories.shape)
print(categories['Type'].value_counts())
print("\nPrimeras filas de 'categories':")
print(categories.head())

DataFrame 'objects':
(15, 6)
Type
art    15
Name: count, dtype: int64

Primeras filas de 'objects':
   index   Component                                        Description Type  \
0     20   Hauff2022  High-performance work practices, employee well...  art   
1     21    Song2021  High involvement work systems and organization...  art   
2     22  Marin 2016  Deconstructing AMO framework: a systematic rev...  art   
3     23   Marin2022  What does the wage structure depend on? Eviden...  art   
4     24   Ahmad2022  The impact of green HRM on green creativity: m...  art   

   Class comments  
0  1hiwp      NaN  
1  1hiwp      NaN  
2  1hiwp      NaN  
3  1hiwp      NaN  
4   2hrn      NaN  


DataFrame 'categories':
(20, 6)
Type
cat1    12
cat4     3
cat5     3
cat2     1
cat3     1
Name: count, dtype: int64

Primeras filas de 'categories':
   index           Component  \
0      0  Select&Recruitment   
1      1            Training   
2      2         Empowerment   
3      3         P


# Embeedings
Quiero hacer tareas de vectorizacion con google collab y dispongo de 12gb de ram (sin GPU) salvo que pase a la versión de pago. La tarea que quiero hacer es vectorizar/embeedings celdas de texto (de unas 200 palabras como máximo cada una). Todos los textos son en inglés. los embbedings los quiero utilizar para una tarea de clasificación de textos cientificos (la entrada es titulo y abstracr en la celda a vectorizar) y tengo varias categorias (con una definicion de la categoria que tambien querría vectorizar) y luego calcular la distancia entre cada articulo y cada categoria



In [6]:
# # Instalar las dependencias necesarias
import pkg_resources
import subprocess

required_packages = [
    'seaborn',
    'matplotlib',
    'sentence-transformers',
    'pandas',
    'numpy',
    'transformers',
    'torch'
]

installed_packages = {pkg.key for pkg in pkg_resources.working_set}

for package in required_packages:
    if package in installed_packages:
        print(f"{package} ya está instalado.")
    else:
        print(f"{package} no está instalado. Instalando...")
        subprocess.check_call(["pip", "install", package])
        print(f"{package} ha sido instalado.")

print("\nTodas las bibliotecas necesarias están ahora instaladas.")


from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import time
import torch
from datetime import datetime

seaborn ya está instalado.
matplotlib ya está instalado.
sentence-transformers no está instalado. Instalando...
sentence-transformers ha sido instalado.
pandas ya está instalado.
numpy ya está instalado.
transformers ya está instalado.
torch ya está instalado.

Todas las bibliotecas necesarias están ahora instaladas.


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
# Obtener la fecha y hora actual para los nombres de archivo
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

In [8]:

# Lista de modelos a probar (con los ejemplos ha llegado a un tope de 6gb de ram)
models_to_test = [
    'all-MiniLM-L6-v2',
    'all-distilroberta-v1',
    'all-mpnet-base-v2',
    'paraphrase-multilingual-mpnet-base-v2',
    'distiluse-base-multilingual-cased-v1',
    'all-MiniLM-L12-v2',
    'allenai-specter',
    'allenai/scibert_scivocab_uncased',
    'distilbert-base-nli-mean-tokens',
    'roberta-base-nli-stsb-mean-tokens',
    'distiluse-base-multilingual-cased-v2',
    'paraphrase-multilingual-MiniLM-L12-v2',
    'stsb-roberta-large',
    'bert-base-nli-mean-tokens'
]

# Función para evaluar un modelo
def evaluate_model(model_name, objects_df, categories_df):
    print(f"Evaluating model: {model_name}")
    model = SentenceTransformer(model_name)

    start_time = time.time()

    # Generar embeddings para objetos
    object_texts = objects_df[target_column].tolist()
    object_embeddings = model.encode(object_texts, show_progress_bar=True)

    # Generar embeddings para categorías
    category_texts = categories_df[target_column].tolist()
    category_embeddings = model.encode(category_texts, show_progress_bar=True)

    # Calcular similitud coseno
    similarity_matrix = util.cos_sim(object_embeddings, category_embeddings)

    # Encontrar las mejores coincidencias
    best_matches = np.argmax(similarity_matrix, axis=1)

    end_time = time.time()
    processing_time = end_time - start_time

    return {
        'model': model_name,
        'best_matches': categories_df.iloc[best_matches]['Component'].tolist(),
        'processing_time': round(processing_time, 2),  # Redondear a 2 decimales
        'embedding_size': object_embeddings.shape[1],
        'similarity_matrix': similarity_matrix,
        'object_embeddings': object_embeddings,
        'category_embeddings': category_embeddings
    }

# Evaluar todos los modelos
results = []
similarity_data = []
embeddings_data = []

for model_name in models_to_test:
    try:
        result = evaluate_model(model_name, objects, categories)
        results.append({
            'model': result['model'],
            'best_matches': result['best_matches'],
            'processing_time': result['processing_time'],
            'embedding_size': result['embedding_size']
        })

        # Preparar datos de similitud para este modelo
        for i, obj in enumerate(objects['Component']):
            row = [model_name, obj] + result['similarity_matrix'][i].tolist()
            similarity_data.append(row)

        # Preparar datos de embeddings para este modelo
        for i, obj in enumerate(objects['Description']):
            embeddings_data.append([model_name, objects['Component'].iloc[i], obj, result['object_embeddings'][i].tolist()])
        for i, cat in enumerate(categories['Description']):
            embeddings_data.append([model_name, categories['Component'].iloc[i], cat, result['category_embeddings'][i].tolist()])

    except Exception as e:
        print(f"Error evaluating {model_name}: {str(e)}")

# Crear DataFrame con resultados
df_results = pd.DataFrame(results)
df_results = df_results.rename(columns={'processing_time': 'processing_time_seconds'})
print(df_results)

# Crear DataFrame de similitud coseno
columns = ['Model', 'Object'] + categories['Component'].tolist()
df_similarity = pd.DataFrame(similarity_data, columns=columns)

# Crear DataFrame de embeddings
df_embeddings = pd.DataFrame(embeddings_data, columns=['Model', 'Component', 'Description', 'Embedding'])

# Crear la nueva tabla de similitud para HIWPshortDescription
df_hiwp_similarity = df_similarity[['Model', 'Object', target_category]].copy()
df_hiwp_similarity.columns = ['Model', 'Object_Description', 'Similarity']
df_hiwp_similarity['Object_Component'] = objects['Component'].tolist() * len(models_to_test)
df_hiwp_similarity = df_hiwp_similarity[['Model', 'Object_Component', 'Object_Description', 'Similarity']]
df_hiwp_similarity = df_hiwp_similarity.sort_values(['Model', 'Similarity'], ascending=[True, False])


Evaluating model: all-MiniLM-L6-v2


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                    model  \
0                        all-MiniLM-L6-v2   
1                    all-distilroberta-v1   
2                       all-mpnet-base-v2   
3   paraphrase-multilingual-mpnet-base-v2   
4    distiluse-base-multilingual-cased-v1   
5                       all-MiniLM-L12-v2   
6                         allenai-specter   
7        allenai/scibert_scivocab_uncased   
8         distilbert-base-nli-mean-tokens   
9       roberta-base-nli-stsb-mean-tokens   
10   distiluse-base-multilingual-cased-v2   
11  paraphrase-multilingual-MiniLM-L12-v2   
12                     stsb-roberta-large   
13              bert-base-nli-mean-tokens   

                                         best_matches  \
0   [HIWPshortDescrip, HIWPshortDescrip, HIWPshort...   
1   [HIWPshortDescrip, HIWPshortDescrip, HIWPLongD...   
2   [HIWPshortDescrip, HIWPshortDescrip, HIWPshort...   
3   [HIWPshortDescrip, HIWPshortDescrip, HIWPLongD...   
4   [HIWPshortDescrip, HIWPshortDescrip

In [ ]:
###OPCIONAL####
# SOLO EJECUTAR para reahacer el analsisi con otra categoria como target sin tener que repetir los embeedings
# el identificador [component] de la categoria que quiero mostrar en el resumen de clasificación global
#target_category ='HIWPshortDescrip'
#target_category ='HIWPLongDescrip'


# Crear la nueva tabla de similitud para HIWPshortDescription
df_hiwp_similarity = df_similarity[['Model', 'Object', target_category]].copy()
df_hiwp_similarity.columns = ['Model', 'Object_Description', 'Similarity']
df_hiwp_similarity['Object_Component'] = objects['Component'].tolist() * len(models_to_test)
df_hiwp_similarity = df_hiwp_similarity[['Model', 'Object_Component', 'Object_Description', 'Similarity']]
df_hiwp_similarity = df_hiwp_similarity.sort_values(['Model', 'Similarity'], ascending=[True, False])




In [9]:
# Calcula el rango de similitud para cada modelo individualmente, donde el rango 1 es el mejor (mayor similitud).
# Calcula el rango promedio para cada Object_Description a través de todos los modelos.
# Ordena los objetos por su rango promedio (el más bajo es el mejor).
# La tabla resumen final contendrá las siguientes columnas:
# # Object_Component: El componente del objeto.
# # Object_Description: La descripción del objeto.
# # Average_Rank: El rango promedio del objeto a través de todos los modelos (más bajo es mejor).
# Los objetos estarán ordenados por su rango promedio,
# lo que  dará una visión integrada de cómo se comportan los objetos a clasificar, a través de todos los modelos,
# en relación con la categoria sobre la que me interesa clasificarlos.

# No es sencillo poner un punto de corte a partir del cual se consideran "excluidos" los objetos a clasificar.
# de modo que se haría un escreening por titulo y abstract manual.
# partiendo del orden de esta tabla hasta llegar aun  punto donde # haya varios seguidos sin seleccionar.
# Momento en el que se podría parar el screening asistido por clasificacion automática.

# Función para calcular el rango inverso (el más alto obtiene el rango 1)
def inverse_rank(series):
    return series.rank(ascending=False, method='min')

# Calcular rangos para cada modelo
df_ranks = df_hiwp_similarity.groupby('Model').apply(lambda x: x.assign(Rank=inverse_rank(x['Similarity']))).reset_index(drop=True)

# Calcular el rango promedio para cada Object_Component a través de todos los modelos
df_summary = df_ranks.groupby('Object_Component')['Rank'].mean().reset_index()
df_summary = df_summary.rename(columns={'Rank': 'Average_Rank'})

# Añadir la Description correspondiente a cada Component
df_summary = df_summary.merge(objects[['Component', 'Description']], left_on='Object_Component', right_on='Component', how='left')

# Ordenar por rango promedio (ascendente, ya que el rango más bajo es mejor)
df_summary = df_summary.sort_values('Average_Rank')

# Reordenar las columnas
df_summary = df_summary[['Object_Component', 'Description', 'Average_Rank']]

# Mostrar las primeras filas de la tabla resumen
print(df_summary.head(20))

       Object_Component                                        Description  \
13             Song2021  High involvement work systems and organization...   
5             Hauff2022  High-performance work practices, employee well...   
1             Ahmad2022  The impact of green HRM on green creativity: m...   
7            Marin 2016  Deconstructing AMO framework: a systematic rev...   
4           Burbano2022  Mitigating Gig and Remote Worker Misconduct: E...   
11             Mora2024  Integrating the SDG into university teaching: ...   
9   Marin2020triplediam  Protocol: Triple Diamond method for problem so...   
3            Becker2022  Surviving remotely: How job control and loneli...   
10            Marin2022  What does the wage structure depend on? Eviden...   
14           rincon2023  The impact of active learning on entrepreneuri...   
0           Aguilar2024  Factors influencing nurse satisfaction with Au...   
8      Marin2020Omcovid  Operations Management at the service of

In [ ]:
# etiquetar los archivos con alguna información adcional
# tag_file = tag_file
# tag_file = 'best7 models'
#tag_file = 'worst7 models'

In [10]:
# código para guardar todos los DataFrames en un único archivo Excel, con cada DataFrame en una hoja separada

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Asegurarse de que la carpeta de salida existe
os.makedirs(output_path, exist_ok=True)

# Crear el nombre del archivo Excel
excel_filename = f'hiwp_all_csv_classification_{current_time}_{target_category}_{tag_file}.xlsx'
excel_path = os.path.join(output_path, excel_filename)

# Crear un ExcelWriter object
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    # Guardar cada DataFrame en una hoja separada
    df_results.to_excel(writer, sheet_name='model_comparison_results', index=False)
    df_similarity.to_excel(writer, sheet_name='tablas_similitud_coseno', index=False)
    df_embeddings.to_excel(writer, sheet_name='embeddings', index=False)
    df_hiwp_similarity.to_excel(writer, sheet_name='hiwp_similarity', index=False)
    df_summary.to_excel(writer, sheet_name='hiwp_similarity_summary', index=False)

print(f"Todos los resultados han sido guardados en {excel_path}")

Todos los resultados han sido guardados en /content/drive/MyDrive/Reto21dias_24/hiwp_all_csv_classification_20240821_153718_HIWPshortDescrip_all models-articles.xlsx


In [ ]:
#si prefiero exportar como csv separado cada resultado
# Guardar
# Asegurarse de que la carpeta de salida existe
os.makedirs(output_path, exist_ok=True)

# # Guardar resultados
# results_path = os.path.join(output_path, f'model_comparison_results_{current_time}_{tag_file}.csv')
# df_results.to_csv(results_path, index=False)
# print(f"Resultados de comparación guardados en {results_path}")

# Guardar DataFrame de similitud coseno
similarity_path = os.path.join(output_path, f'tablas_similitud_coseno_{current_time}_{tag_file}.csv')
df_similarity.to_csv(similarity_path, index=False)
print(f"Tablas de similitud coseno guardadas en {similarity_path}")

# Guardar DataFrame de embeddings
embeddings_path = os.path.join(output_path, f'embeddings_{current_time}_{tag_file}.csv')
df_embeddings.to_csv(embeddings_path, index=False)
print(f"Embeddings guardados en {embeddings_path}")

# # guardar la nueva tabla de similitud para HIWPshortDescription
# hiwp_similarity_path = os.path.join(output_path, f'hiwp_similarity_{current_time}_{tag_file}.csv')
# df_hiwp_similarity.to_csv(hiwp_similarity_path, index=False)
# print(f"Tabla de similitud para HIWPshortDescrip guardada en {hiwp_similarity_path}")

# # Guardar la tabla resumen
# summary_path = os.path.join(output_path, f'hiwp_similarity_summary_{current_time}_{tag_file}.csv')
# df_summary.to_csv(summary_path, index=False)
# print(f"Tabla resumen de similitud para HIWPshortDescription guardada en {summary_path}")

Tablas de similitud coseno guardadas en /content/drive/MyDrive/Reto21dias_24/tablas_similitud_coseno_20240815_100816_all models.csv
Embeddings guardados en /content/drive/MyDrive/Reto21dias_24/embeddings_20240815_100816_all models.csv


In [ ]:
# liberar ram si se va acumulando variables en memoria
import gc
gc.collect()

18